In [20]:
import argparse, os, io
import re, csv
import sagemaker, boto3
from sagemaker.mxnet.model import MXNetModel
import logging
logging.getLogger().setLevel(logging.WARNING)

from scipy import io as sio
import numpy as np

from sagemaker.tensorflow.model import TensorFlowModel

import cv2

In [12]:
sagemaker_model = TensorFlowModel(
    role='arn:aws:iam::821488635735:role/service-role/AmazonSageMaker-ExecutionRole-20171216T132860',
        model_data='s3://sagemaker-us-east-2-821488635735/object_detection/model.tar.gz',
        entry_point='object_detect.py')

In [13]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                       instance_type='ml.m4.xlarge')
print ("Model Loaded, Now start predicting")

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2018-07-01-16-16-30-265
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2018-07-01-16-16-30-265


----------------------------------------------!Model Loaded, Now start predicting


In [38]:
# load the data 
data = sio.loadmat('nyu_training_data.mat')


In [39]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Names', 'images', 'labels'])

In [40]:
images = data['images']
labels = data['labels']
names = data['Names']

In [44]:
img = images[0, :, :, :]
im = cv2.resize(img, (80, 80))
lab = labels[0, :]

In [45]:
print(im.shape)

(80, 80, 3)


In [95]:
def create_names_list(names):
    nms = []
    for name in names:
        nms.append(name[0][0])
    return nms

In [81]:
def predict_data(im, predictor, lab):
    pred_out = predictor.predict(data=np.float32(im)/255.)
    
    probs = np.array(pred_out['outputs']['classes']['floatVal'])
    
    nms = names[probs >= 0.5]
    return nms

In [105]:
pos = 0
neg = 0
for idx in range(images.shape[0]):
    img = images[idx, :, :, :]
    im = cv2.resize(img, (80, 80))
    
    lab = labels[idx, :]
    
    nms = predict_data(im, predictor, lab)
    nms = create_names_list(nms)
    actual = names[lab == 1.]
    actual = create_names_list(actual)
    cur_pos = 0
    for nm in nms:
        for act in actual:
            if nm == act:
                pos+=1
                cur_pos+=1
    neg+=(len(actual) - cur_pos)
                
    

In [108]:
acc = pos *1.0 / (pos + neg)

In [109]:
print('predicted positive samples are : ' + str(pos))
print('predicted negative samples are : ' + str(neg))
print('predicted negative samples are : ' + str(acc))

predicted positive samples are : 210
predicted negative samples are : 304
predicted negative samples are : 0.4085603112840467


0.4085603112840467